In [1]:
import sys
sys.path.append("..")

In [2]:
import bytelevel

In [3]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [4]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import numpy as np

In [5]:
from keras.layers import Embedding, Conv1D, Input, GRU, LSTM, Bidirectional, Dense
from keras.models import Model

In [6]:
r = np.random.RandomState(42)

In [7]:
r = np.random.RandomState(42)

def random_chop(s, r, m):
    n = len(s)
    if n <= m:
        return s
    k = r.randint(n - m)
    s = s[k:]
    return s[:m]

In [8]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

In [9]:
max_len = 50

def dataset(x):
    x = [random_chop(s, r, max_len + 1) for s in x]
    x = bytelevel.encode(x)
    x = pad_sequences(x, max_len + 1)
    return x

x_train = dataset(train['data'])
x_test = dataset(test['data'])
x_test = x_test[:2000]

In [33]:
class EmbeddingHyper(object):
    def __init__(self, vocab_size, embedding_dim):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
    @staticmethod
    def Random(r):
        embedding_dim = r.choice([2 ** i for i in range(4, 10)])
        return EmbeddingHyper(256, embedding_dim)
        
    def display(self):
        print("embedding")
        print("vocab size=%d" % self.vocab_size)
        print("embedding dimension=%d" % self.embedding_dim)
        
    def make_layer(self, name='embedder'):
        return Embedding(self.vocab_size, 
            self.embedding_dim , name=name)
    
class ConvHyper(object):
    def __init__(self, filters, kernel_size=3, stride=2):
        self.filters = filters
        self.kernel_size = kernel_size
        self.stride = stride
        
    @staticmethod
    def Random(r):
        filters = r.choice([2 ** i for i in range(6, 10)])
        kernel_size = r.randint(4) + 2
        stride = r.randint(4) + 1
        return ConvHyper(filters, kernel_size, stride)
        
    def display(self):
        print("conv 1d")
        print("filters=%d" % self.filters)
        print("kernel size=%d" % self.kernel_size)
        print("stride = %d" % self.stride)
        
    def make_layer(self, name):
        return Conv1D(64, 4, strides=2, activation='relu', name=name)
    
class RnnHyper(object):
    def __init__(self, hidden_dim, is_lstm, is_bidirectional):
        self.hidden_dim = hidden_dim
        self.is_lstm = is_lstm
        self.is_bidirectional = is_bidirectional
        
    @staticmethod
    def Random(r):
        hidden_dim = r.choice([2 ** i for i in range(6, 10)])
        is_lstm = bool(r.randint(2))
        is_bidirectional = bool(r.randint(2))
        return RnnHyper(hidden_dim, is_lstm, is_bidirectional)

    def display(self):
        print("RNN")
        print("hidden dimension=%d" % self.hidden_dim)
        if self.is_bidirectional:
            print("bidirectional")
        if self.is_lstm:
            print("lstm")
        else:
            print("gru")            
        
    def make_layer(self, name, return_sequences):
        if self.is_lstm:
            make_rnn = LSTM
        else:
            make_rnn = GRU
        if self.is_bidirectional:
            return Bidirectional(make_rnn(self.hidden_dim, return_sequences=return_sequences), name=name)
        return make_rnn(self.hidden_dim, return_sequences=return_sequences, name=name)
    

In [16]:
embedd_h = EmbeddingHyper(256, 64)
conv_h = ConvHyper(512)
rnn_h = RnnHyper(512, is_lstm=True, is_bidirectional=True)

embedd_h.display()
print()
conv_h.display()
print()
rnn_h.display()
print()

embed = embedd_h.make_layer()
c1d0 = conv_h.make_layer('conv0')
rnn = rnn_h.make_layer('rnn', return_sequences=False)
pred = Dense(256, activation='softmax')

embedding
vocab size=256
embedding dimension=64

conv 1d
filters=512
kernel size=3
stride = 2

RNN
hidden dimension=512
bidirectional
lstm



In [12]:
x = Input(shape=(None,), name='text_input')
h = embed(x)
print(h.shape)
h = c1d0(h)
print(h.shape)
h = rnn(h)
print(h.shape)
h = pred(h)
print(h.shape)

model = Model(x, h)
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'])


(?, ?, 64)
(?, ?, 64)
(?, 1024)
(?, 256)


In [14]:
model.fit(x=x_train[:, :max_len], 
                 y=x_train[:, max_len],
                epochs=10, batch_size=10,
                validation_data=(x_test[:, :max_len], x_test[:, max_len]))

Train on 11314 samples, validate on 2000 samples
Epoch 1/10
11314/11314 [==============================] - 141s - loss: 2.4974 - sparse_categorical_accuracy: 0.3192 - val_loss: 2.6302 - val_sparse_categorical_accuracy: 0.3055
Epoch 2/10
11314/11314 [==============================] - 142s - loss: 2.2949 - sparse_categorical_accuracy: 0.3621 - val_loss: 2.6245 - val_sparse_categorical_accuracy: 0.3185
Epoch 3/10
11314/11314 [==============================] - 142s - loss: 2.0331 - sparse_categorical_accuracy: 0.4166 - val_loss: 2.6610 - val_sparse_categorical_accuracy: 0.3280
Epoch 4/10
11314/11314 [==============================] - 142s - loss: 1.6544 - sparse_categorical_accuracy: 0.5162 - val_loss: 2.7765 - val_sparse_categorical_accuracy: 0.3160
Epoch 5/10
11314/11314 [==============================] - 141s - loss: 1.1616 - sparse_categorical_accuracy: 0.6625 - val_loss: 3.0024 - val_sparse_categorical_accuracy: 0.3170
Epoch 6/10
11314/11314 [==============================] - 142s - l

In [39]:
r = np.random.RandomState(42)

for _ in range(20):
    embedd_h = EmbeddingHyper.Random(r)
    embedd_h.display()
    print()

    conv_h = ConvHyper.Random(r)
    conv_h.display()
    print()

    rnn_h = RnnHyper.Random(r)
    rnn_h.display()
    print()

    embed = embedd_h.make_layer()
    c1d0 = conv_h.make_layer('conv0')
    rnn = rnn_h.make_layer('rnn', return_sequences=False)
    pred = Dense(256, activation='softmax')
    
    x = Input(shape=(None,), name='text_input')
    h = embed(x)
    print(h.shape)
    h = c1d0(h)
    print(h.shape)
    h = rnn(h)
    print(h.shape)
    h = pred(h)
    print(h.shape)

    model = Model(x, h)
    model.compile(optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'])

    model.fit(x=x_train[:, :max_len], 
                 y=x_train[:, max_len],
                epochs=3, batch_size=10,
                validation_data=(x_test[:, :max_len], x_test[:, max_len]))
    
    print()
    print()
    print()
    

embedding
vocab size=256
embedding dimension=128

conv 1d
filters=64
kernel size=4
stride = 3

RNN
hidden dimension=512
gru

(?, ?, 128)
(?, ?, 64)
(?, 512)
(?, 256)
Train on 11314 samples, validate on 2000 samples
Epoch 1/3
11314/11314 [==============================] - 52s - loss: 3.1687 - sparse_categorical_accuracy: 0.2134 - val_loss: 2.8369 - val_sparse_categorical_accuracy: 0.2695
Epoch 2/3
11314/11314 [==============================] - 52s - loss: 2.7042 - sparse_categorical_accuracy: 0.2911 - val_loss: 2.6936 - val_sparse_categorical_accuracy: 0.2835
Epoch 3/3
11314/11314 [==============================] - 52s - loss: 2.4855 - sparse_categorical_accuracy: 0.3314 - val_loss: 2.5910 - val_sparse_categorical_accuracy: 0.3205



embedding
vocab size=256
embedding dimension=32

conv 1d
filters=256
kernel size=4
stride = 3

RNN
hidden dimension=256
lstm

(?, ?, 32)
(?, ?, 64)
(?, 256)
(?, 256)
Train on 11314 samples, validate on 2000 samples
Epoch 1/3
11314/11314 [===================

11314/11314 [==============================] - 112s - loss: 2.6239 - sparse_categorical_accuracy: 0.3030 - val_loss: 2.6682 - val_sparse_categorical_accuracy: 0.3000



embedding
vocab size=256
embedding dimension=128

conv 1d
filters=64
kernel size=5
stride = 3

RNN
hidden dimension=256
lstm

(?, ?, 128)
(?, ?, 64)
(?, 256)
(?, 256)
Train on 11314 samples, validate on 2000 samples
Epoch 1/3
11314/11314 [==============================] - 31s - loss: 3.2253 - sparse_categorical_accuracy: 0.2118 - val_loss: 2.8733 - val_sparse_categorical_accuracy: 0.2525
Epoch 2/3
11314/11314 [==============================] - 30s - loss: 2.7543 - sparse_categorical_accuracy: 0.2791 - val_loss: 2.6910 - val_sparse_categorical_accuracy: 0.2905
Epoch 3/3
11314/11314 [==============================] - 30s - loss: 2.5694 - sparse_categorical_accuracy: 0.3121 - val_loss: 2.6318 - val_sparse_categorical_accuracy: 0.3135



embedding
vocab size=256
embedding dimension=128

conv 1d
filters=128
kernel size=5
str

In [43]:
class EmbeddingHyper(object):
    def __init__(self, vocab_size, embedding_dim):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
    @staticmethod
    def Random(r):
        embedding_dim = r.choice([2 ** i for i in range(7, 9)])
        return EmbeddingHyper(256, embedding_dim)
        
    def display(self):
        print("embedding")
        print("vocab size=%d" % self.vocab_size)
        print("embedding dimension=%d" % self.embedding_dim)
        
    def make_layer(self, name='embedder'):
        return Embedding(self.vocab_size, 
            self.embedding_dim , name=name)
    
class ConvHyper(object):
    def __init__(self, filters, kernel_size=3, stride=2):
        self.filters = filters
        self.kernel_size = kernel_size
        self.stride = stride
        
    @staticmethod
    def Random(r):
        filters = r.choice([2 ** i for i in range(6, 10)])
        kernel_size = r.randint(4) + 4
        stride = r.randint(4) + 3
        return ConvHyper(filters, kernel_size, stride)
        
    def display(self):
        print("conv 1d")
        print("filters=%d" % self.filters)
        print("kernel size=%d" % self.kernel_size)
        print("stride = %d" % self.stride)
        
    def make_layer(self, name):
        return Conv1D(64, 4, strides=2, activation='relu', name=name)
    
class RnnHyper(object):
    def __init__(self, hidden_dim, is_lstm, is_bidirectional):
        self.hidden_dim = hidden_dim
        self.is_lstm = is_lstm
        self.is_bidirectional = is_bidirectional
        
    @staticmethod
    def Random(r):
        hidden_dim = r.choice([2 ** i for i in range(6, 8)])
        is_lstm = bool(r.randint(2))
        is_bidirectional = bool(r.randint(2))
        return RnnHyper(hidden_dim, is_lstm, is_bidirectional)

    def display(self):
        print("RNN")
        print("hidden dimension=%d" % self.hidden_dim)
        if self.is_bidirectional:
            print("bidirectional")
        if self.is_lstm:
            print("lstm")
        else:
            print("gru")            
        
    def make_layer(self, name, return_sequences):
        if self.is_lstm:
            make_rnn = LSTM
        else:
            make_rnn = GRU
        if self.is_bidirectional:
            return Bidirectional(make_rnn(self.hidden_dim, return_sequences=return_sequences), name=name)
        return make_rnn(self.hidden_dim, return_sequences=return_sequences, name=name)
    


In [44]:
r = np.random.RandomState(43)

for _ in range(20):
    embedd_h = EmbeddingHyper.Random(r)
    embedd_h.display()
    print()

    conv_h = ConvHyper.Random(r)
    conv_h.display()
    print()

    rnn_h = RnnHyper.Random(r)
    rnn_h.display()
    print()

    embed = embedd_h.make_layer()
    c1d0 = conv_h.make_layer('conv0')
    rnn = rnn_h.make_layer('rnn', return_sequences=False)
    pred = Dense(256, activation='softmax')
    
    x = Input(shape=(None,), name='text_input')
    h = embed(x)
    print(h.shape)
    h = c1d0(h)
    print(h.shape)
    h = rnn(h)
    print(h.shape)
    h = pred(h)
    print(h.shape)

    model = Model(x, h)
    model.compile(optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'])

    model.fit(x=x_train[:, :max_len], 
                 y=x_train[:, max_len],
                epochs=3, batch_size=10,
                validation_data=(x_test[:, :max_len], x_test[:, max_len]))
    
    print()
    print()
    print()
    

embedding
vocab size=256
embedding dimension=64

conv 1d
filters=64
kernel size=7
stride = 4

RNN
hidden dimension=128
gru

(?, ?, 64)
(?, ?, 64)
(?, 128)
(?, 256)
Train on 11314 samples, validate on 2000 samples
Epoch 1/3
11314/11314 [==============================] - 22s - loss: 3.2328 - sparse_categorical_accuracy: 0.2043 - val_loss: 2.8495 - val_sparse_categorical_accuracy: 0.2580
Epoch 2/3
11314/11314 [==============================] - 21s - loss: 2.7598 - sparse_categorical_accuracy: 0.2813 - val_loss: 2.7257 - val_sparse_categorical_accuracy: 0.2805
Epoch 3/3
11314/11314 [==============================] - 21s - loss: 2.5856 - sparse_categorical_accuracy: 0.3099 - val_loss: 2.6567 - val_sparse_categorical_accuracy: 0.3155



embedding
vocab size=256
embedding dimension=128

conv 1d
filters=512
kernel size=7
stride = 5

RNN
hidden dimension=64
bidirectional
gru

(?, ?, 128)
(?, ?, 64)
(?, 128)
(?, 256)
Train on 11314 samples, validate on 2000 samples
Epoch 1/3
11314/11314 [=======

11314/11314 [==============================] - 38s - loss: 2.4322 - sparse_categorical_accuracy: 0.3429 - val_loss: 2.5553 - val_sparse_categorical_accuracy: 0.3315



embedding
vocab size=256
embedding dimension=256

conv 1d
filters=256
kernel size=6
stride = 3

RNN
hidden dimension=64
bidirectional
lstm

(?, ?, 256)
(?, ?, 64)
(?, 128)
(?, 256)
Train on 11314 samples, validate on 2000 samples
Epoch 1/3
11314/11314 [==============================] - 47s - loss: 3.3012 - sparse_categorical_accuracy: 0.1967 - val_loss: 2.9655 - val_sparse_categorical_accuracy: 0.2450
Epoch 2/3
11314/11314 [==============================] - 45s - loss: 2.8264 - sparse_categorical_accuracy: 0.2726 - val_loss: 2.7762 - val_sparse_categorical_accuracy: 0.2635
Epoch 3/3
11314/11314 [==============================] - 45s - loss: 2.6510 - sparse_categorical_accuracy: 0.3048 - val_loss: 2.6887 - val_sparse_categorical_accuracy: 0.3035



embedding
vocab size=256
embedding dimension=256

conv 1d
filters=64
kerne

In [45]:
class EmbeddingHyper(object):
    def __init__(self, vocab_size, embedding_dim):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
    @staticmethod
    def Random(r):
        embedding_dim = r.choice([2 ** i for i in range(7, 9)])
        return EmbeddingHyper(256, embedding_dim)
        
    def display(self):
        print("embedding")
        print("vocab size=%d" % self.vocab_size)
        print("embedding dimension=%d" % self.embedding_dim)
        
    def make_layer(self, name='embedder'):
        return Embedding(self.vocab_size, 
            self.embedding_dim , name=name)
    
class ConvHyper(object):
    def __init__(self, filters, kernel_size=3, stride=2):
        self.filters = filters
        self.kernel_size = kernel_size
        self.stride = stride
        
    @staticmethod
    def Random(r):
        filters = r.choice([2 ** i for i in range(7, 9)])
        kernel_size = r.randint(4) + 4
        stride = r.randint(4) + 3
        return ConvHyper(filters, kernel_size, stride)
        
    def display(self):
        print("conv 1d")
        print("filters=%d" % self.filters)
        print("kernel size=%d" % self.kernel_size)
        print("stride = %d" % self.stride)
        
    def make_layer(self, name):
        return Conv1D(64, 4, strides=2, activation='relu', name=name)
    
class RnnHyper(object):
    def __init__(self, hidden_dim, is_lstm, is_bidirectional):
        self.hidden_dim = hidden_dim
        self.is_lstm = is_lstm
        self.is_bidirectional = is_bidirectional
        
    @staticmethod
    def Random(r):
        hidden_dim = r.choice([2 ** i for i in range(7, 9)])
        is_lstm = bool(r.randint(2))
        is_bidirectional = bool(r.randint(2))
        return RnnHyper(hidden_dim, is_lstm, is_bidirectional)

    def display(self):
        print("RNN")
        print("hidden dimension=%d" % self.hidden_dim)
        if self.is_bidirectional:
            print("bidirectional")
        if self.is_lstm:
            print("lstm")
        else:
            print("gru")            
        
    def make_layer(self, name, return_sequences):
        if self.is_lstm:
            make_rnn = LSTM
        else:
            make_rnn = GRU
        if self.is_bidirectional:
            return Bidirectional(make_rnn(self.hidden_dim, return_sequences=return_sequences), name=name)
        return make_rnn(self.hidden_dim, return_sequences=return_sequences, name=name)
    


In [46]:
np.mean([len(s) for s in train['data']])

1949.3100583348064

In [47]:
max_len = 256

def dataset(x):
    x = [random_chop(s, r, max_len + 1) for s in x]
    x = bytelevel.encode(x)
    x = pad_sequences(x, max_len + 1)
    return x

x_train = dataset(train['data'])
x_test = dataset(test['data'])
x_test = x_test[:2000]

In [48]:
def hyper_search(seed):
    r = np.random.RandomState(seed)

    for _ in range(20):
        embedd_h = EmbeddingHyper.Random(r)
        embedd_h.display()
        print()

        conv_h = ConvHyper.Random(r)
        conv_h.display()
        print()

        rnn_h = RnnHyper.Random(r)
        rnn_h.display()
        print()

        embed = embedd_h.make_layer()
        c1d0 = conv_h.make_layer('conv0')
        rnn = rnn_h.make_layer('rnn', return_sequences=False)
        pred = Dense(256, activation='softmax')

        x = Input(shape=(None,), name='text_input')
        h = embed(x)
        print(h.shape)
        h = c1d0(h)
        print(h.shape)
        h = rnn(h)
        print(h.shape)
        h = pred(h)
        print(h.shape)

        model = Model(x, h)
        model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['sparse_categorical_accuracy'])

        model.fit(x=x_train[:, :max_len], 
                     y=x_train[:, max_len],
                    epochs=3, batch_size=10,
                    validation_data=(x_test[:, :max_len], x_test[:, max_len]))

        print()
        print()
        print()

In [49]:
hyper_search(44)

embedding
vocab size=256
embedding dimension=128

conv 1d
filters=256
kernel size=5
stride = 4

RNN
hidden dimension=256
lstm

(?, ?, 128)
(?, ?, 64)
(?, 256)
(?, 256)
Train on 11314 samples, validate on 2000 samples
Epoch 1/3
11314/11314 [==============================] - 135s - loss: 3.0896 - sparse_categorical_accuracy: 0.2314 - val_loss: 2.7451 - val_sparse_categorical_accuracy: 0.2905
Epoch 2/3
11314/11314 [==============================] - 133s - loss: 2.6465 - sparse_categorical_accuracy: 0.3018 - val_loss: 2.6434 - val_sparse_categorical_accuracy: 0.3125
Epoch 3/3
11314/11314 [==============================] - 134s - loss: 2.5065 - sparse_categorical_accuracy: 0.3273 - val_loss: 2.5655 - val_sparse_categorical_accuracy: 0.3295



embedding
vocab size=256
embedding dimension=128

conv 1d
filters=256
kernel size=7
stride = 5

RNN
hidden dimension=128
bidirectional
lstm

(?, ?, 128)
(?, ?, 64)
(?, 256)
(?, 256)
Train on 11314 samples, validate on 2000 samples
Epoch 1/3
11314/11314

11314/11314 [==============================] - 237s - loss: 2.5150 - sparse_categorical_accuracy: 0.3250 - val_loss: 2.5691 - val_sparse_categorical_accuracy: 0.3270



embedding
vocab size=256
embedding dimension=128

conv 1d
filters=256
kernel size=5
stride = 6

RNN
hidden dimension=128
bidirectional
gru

(?, ?, 128)
(?, ?, 64)
(?, 256)
(?, 256)
Train on 11314 samples, validate on 2000 samples
Epoch 1/3
11314/11314 [==============================] - 198s - loss: 3.1304 - sparse_categorical_accuracy: 0.2270 - val_loss: 2.7433 - val_sparse_categorical_accuracy: 0.2985
Epoch 2/3
11314/11314 [==============================] - 195s - loss: 2.6158 - sparse_categorical_accuracy: 0.3075 - val_loss: 2.6015 - val_sparse_categorical_accuracy: 0.3190
Epoch 3/3
11314/11314 [==============================] - 195s - loss: 2.4450 - sparse_categorical_accuracy: 0.3380 - val_loss: 2.5445 - val_sparse_categorical_accuracy: 0.3365



embedding
vocab size=256
embedding dimension=256

conv 1d
filters=256


In [92]:
maxlen = 72
x = Input(shape=(None,), name='text_input')
embed = Embedding(256, 
            64, name='embedder')
c1d0 = Conv1D(256, 4, strides=2, activation='relu')
c1d1 = Conv1D(256, 4, strides=2, activation='relu')
rnn = Bidirectional(GRU(512, return_sequences=False))
pred = Dense(256, activation='softmax')

h = embed(x)
print(h.shape)
h = c1d0(h)
print(h.shape)
h = c1d1(h)
print(h.shape)
h = rnn(h)
print(h.shape)
h = pred(h)
print(h.shape)

model = Model(x, h)
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'])

model.fit(x=x_train[:, :72], 
                 y=x_train[:, 72],
                epochs=10, batch_size=2,
                validation_data=(x_test[:, :72], x_test[:, 72]))

(?, ?, 64)
(?, ?, 256)
(?, ?, 256)
(?, 1024)
(?, 256)
Train on 11314 samples, validate on 2000 samples
Epoch 1/10
11314/11314 [==============================] - 400s - loss: 3.5409 - sparse_categorical_accuracy: 0.1640 - val_loss: 3.4404 - val_sparse_categorical_accuracy: 0.1610
Epoch 2/10
11314/11314 [==============================] - 406s - loss: 3.4104 - sparse_categorical_accuracy: 0.1707 - val_loss: 3.3954 - val_sparse_categorical_accuracy: 0.1655
Epoch 3/10
11314/11314 [==============================] - 408s - loss: 3.3576 - sparse_categorical_accuracy: 0.1754 - val_loss: 3.3681 - val_sparse_categorical_accuracy: 0.1715
Epoch 4/10
11314/11314 [==============================] - 410s - loss: 3.2890 - sparse_categorical_accuracy: 0.1795 - val_loss: 3.3671 - val_sparse_categorical_accuracy: 0.1710
Epoch 5/10
11314/11314 [==============================] - 410s - loss: 3.2725 - sparse_categorical_accuracy: 0.1799 - val_loss: 3.4650 - val_sparse_categorical_accuracy: 0.1730
Epoch 6/10
1